* Generate smaples from forecast everyday
* Augument positive samples by using a sliding window
* filename convention: 
    * real_day498_pos_lat_lon_mag0
    * aug_day498_pos_lat_lon_mag0


In [1]:
import sys
from glob import glob

import time
import h5py
import zarr
import numpy as np
import pandas as pd
from scipy.spatial import cKDTree

In [2]:
sys.path.insert(0, '/glade/u/home/ksha/NCAR/')
sys.path.insert(0, '/glade/u/home/ksha/NCAR/libs/')

from namelist import *
import data_utils as du

In [3]:
from datetime import datetime, timedelta
import subprocess

In [30]:
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
means = [
    -6.335041783675384,
    101598.30648208999,
    2.4340308170812857,
    0.0238316214287872,
    0.0115228964831135,
    0.015723252607236175,
    0.00010298927478466365,
    0.00013315081911787703,
    0.02022990418418194,
    285.1588453352469,
    280.69456763975046,
    0.18025322895802864,
    -0.35625256772098957,
    4.466962100212334,
    0.10710428466431396,
    311.51020050786116,
    -22.95554152474839,
    95.80303950026172,
    41.22773039479408,
    2.696538199313979,
    0.257023643073863,
    11.80181492281666,
    0.15778718430103703,
];

stds = [
    8.872575669978966,
    672.3339463894478,
    7.555104640235371,
    0.5696550725786566,
    0.2283199203388272,
    0.37333362094670486,
    0.00022281640603195643,
    0.0002413561909874066,
    0.3589573748563584,
    11.553795616392204,
    12.101590155483459,
    3.1758721705443826,
    3.6588052023281175,
    2.6995797278745948,
    0.9896017905552607,
    748.8376068157106,
    78.895180023938,
    104.17948262883918*2,
    77.25788246299936*2,
    5.35086729614372,
    5.438075471238217,
    11.440203318938076,
    11.327741531273508
];

ind_pick = [0, 1, 3, 4, 5, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]

log_norm = [True, False, True, True, True, True, True, True, True, False, False, 
            False, False, True, True, True, True, False, False, False, False, False, False]

sparse = [True, False, True, True, True, True, True, True, True, False, False, 
          False, False, False, True, True, True, False, False, False, False, False, False]


In [5]:
lead = 9

In [6]:
print('Generating batches from lead{}'.format(lead))

#HRRRv3_lead = zarr.load(save_dir_scratch+'HRRR_{:02}_v3.zarr'.format(lead))
HRRRv3_lead = np.zeros((872, 1059, 1799, 23)) # use fake data for a quick test
    
with h5py.File(save_dir+'HRRR_domain.hdf', 'r') as h5io:
    lon_3km = h5io['lon_3km'][...]
    lat_3km = h5io['lat_3km'][...]
    lon_72km = h5io['lon_72km'][...]
    lat_72km = h5io['lat_72km'][...]
    
with h5py.File(save_dir+'HRRR_torn_monthly.hdf', 'r') as h5io:
    clim = h5io['clim'][:]

Generating batches from lead9


In [15]:
def neighbour_leads(lead):
    out = [lead-2, lead-1, lead, lead+1, lead+2]
    flag_shift = [0, 0, 0, 0, 0]
    
    for i in range(5):
        if out[i] < 0:
            out[i] = 24+out[i]
            flag_shift[i] = -1
        if out[i] > 23:
            out[i] = out[i]-24
            flag_shift[i] = +1
            
    return out, flag_shift
        
    

In [17]:
lead_window, flag_shift = neighbour_leads(lead)

In [50]:
record_all = ()

for i, lead_temp in enumerate(lead_window):
    
    flag_ = flag_shift[i]
    
    with h5py.File(save_dir_scratch+'SPC_to_lead{}_72km.hdf'.format(lead_temp), 'r') as h5io:
        record_temp = h5io['record_v3'][...]
        
    if flag_shift[i] == 0:
        record_all = record_all + (record_temp,)
        
    if flag_shift[i] == -1:
        record_temp[1:, ...] = record_temp[:-1, ...]
        record_temp[0, ...] = np.nan
        record_all = record_all + (record_temp,)
    
    if flag_shift[i] == +1:
        record_temp[:-1, ...] = record_temp[1:, ...]
        record_temp[-1, ...] = np.nan
        record_all = record_all + (record_temp,)


shape_record = record_temp.shape      
record_v3 = np.empty(shape_record)
record_v3[...] = np.nan

for i in range(5):
    record_temp = record_all[i]
    for day in range(shape_record[0]):
        for ix in range(shape_record[1]):
            for iy in range(shape_record[2]):
                if np.logical_not(np.isnan(record_temp[day, ix, iy])):
                    record_v3[day, ix, iy] = record_temp[day, ix, iy]
        

In [52]:
np.sum(np.logical_not(np.isnan(record_v3)))#.shape

206

In [7]:
record_v3.shape

(872, 44, 75)

In [8]:
shape_72km = lon_72km.shape
shape_3km = lon_3km.shape
shape_record = record_v3.shape

In [9]:
indx_array = np.empty(shape_72km)
indy_array = np.empty(shape_72km)

gridTree = cKDTree(list(zip(lon_3km.ravel(), lat_3km.ravel()))) #KDTree_wraper(xgrid, ygrid)

for xi in range(shape_72km[0]):
    for yi in range(shape_72km[1]):
        
        temp_lon = lon_72km[xi, yi]
        temp_lat = lat_72km[xi, yi]
        
        dist, indexes = gridTree.query(list(zip(np.array(temp_lon)[None], np.array(temp_lat)[None])))
        indx_3km, indy_3km = np.unravel_index(indexes, shape_3km)
        
        indx_array[xi, yi] = indx_3km[0]
        indy_array[xi, yi] = indy_3km[0]

In [10]:
base_v3_s = datetime(2018, 7, 15)
base_v3_e = datetime(2020, 12, 2)

base_v4_s = datetime(2020, 12, 3)
base_v4_e = datetime(2022, 7, 15)

base_ref = datetime(2010, 1, 1)

date_list_v3 = [base_v3_s + timedelta(days=day) for day in range(365+365+142)]
date_list_v4 = [base_v4_s + timedelta(days=day) for day in range(365+180)]

In [11]:
L_train = shape_record[0]

input_size = 128
half_margin = 64
L_vars = len(ind_pick)
out_slice = np.empty((1, input_size, input_size, L_vars))

In [12]:
batch_dir = '/glade/scratch/ksha/DATA/NCAR_batch/'
prefix = 'real_day{}_{}_indx{}_indy{}_mag{}_lead{}.npy'

In [54]:
for day in range(L_train):
    for ix in range(shape_72km[0]):
        for iy in range(shape_72km[1]):
            
            indx = int(indx_array[ix, iy])
            indy = int(indy_array[ix, iy])
            
            x_edge_left = indx - half_margin
            x_edge_right = indx + half_margin

            y_edge_bottom = indy - half_margin
            y_edge_top = indy + half_margin
            
            if x_edge_left >= 0 and y_edge_bottom >= 0 and x_edge_right <= shape_3km[0] and y_edge_top <= shape_3km[1]:
                
                hrrr_3km = HRRRv3_lead[day, x_edge_left:x_edge_right, y_edge_bottom:y_edge_top, :]
                
                for v, ind_var in enumerate(ind_pick):
                    temp = hrrr_3km[..., ind_var]
                    if ind_var == 0:
                        temp[temp<0] = 0
                    if log_norm[ind_var]:
                        temp = np.log(np.abs(temp)+1)
                    else:
                        temp = (temp - means[ind_var])/stds[ind_var]
                            
                    out_slice[..., v] = temp
                
                obs_temp = record_v3[day, ix, iy]
            
                if np.isnan(obs_temp):
                    flag_torn = 'neg'
                else:
                    flag_torn = 'pos'
                    obs_temp = int(obs_temp)
                    
                if np.sum(np.isnan(out_slice)) > 0:
                    print('HRRR contains NaN')
                    continue;
                else:
                    save_name = batch_dir+prefix.format(day, flag_torn, ix, iy, obs_temp, lead)
                    print(save_name)
                    #np.save(save_name, out_slice)

/glade/scratch/ksha/DATA/NCAR_batch/real_day0_neg_indx3_indy3_magnan_lead9.npy
/glade/scratch/ksha/DATA/NCAR_batch/real_day0_neg_indx3_indy4_magnan_lead9.npy
/glade/scratch/ksha/DATA/NCAR_batch/real_day0_neg_indx3_indy5_magnan_lead9.npy
/glade/scratch/ksha/DATA/NCAR_batch/real_day0_neg_indx3_indy6_magnan_lead9.npy
/glade/scratch/ksha/DATA/NCAR_batch/real_day0_neg_indx3_indy7_magnan_lead9.npy
/glade/scratch/ksha/DATA/NCAR_batch/real_day0_neg_indx3_indy8_magnan_lead9.npy
/glade/scratch/ksha/DATA/NCAR_batch/real_day0_neg_indx3_indy9_magnan_lead9.npy
/glade/scratch/ksha/DATA/NCAR_batch/real_day0_neg_indx3_indy10_magnan_lead9.npy
/glade/scratch/ksha/DATA/NCAR_batch/real_day0_neg_indx3_indy11_magnan_lead9.npy
/glade/scratch/ksha/DATA/NCAR_batch/real_day0_neg_indx3_indy12_magnan_lead9.npy
/glade/scratch/ksha/DATA/NCAR_batch/real_day0_neg_indx3_indy13_magnan_lead9.npy
/glade/scratch/ksha/DATA/NCAR_batch/real_day0_neg_indx3_indy14_magnan_lead9.npy
/glade/scratch/ksha/DATA/NCAR_batch/real_day0_n

KeyboardInterrupt: 

In [26]:
x_edge_left

19.0